# Sequence Optimization Demo: Disordered, Self-Interacting, and Non-Interacting Sequence

This notebook demonstrates how to use the `SequenceOptimizer` class to generate a protein sequence that:
- Is predicted to be fully disordered (using `FractionDisorder`)
- Interacts strongly with itself (using `MeanSelfEpsilon`)
- Does **not** interact with a specified target sequence (using `MeanEpsilonWithTarget`)

We will use the `goose` package and its optimization backend for this demonstration.

In [ ]:
# Import required libraries
import goose
from goose.optimize import SequenceOptimizer
from sparrow.protein import Protein
import numpy as np
import metapredict as meta

# For reproducibility
import random
random.seed(42)
np.random.seed(42)

## Setup: Define Parameters and Target Sequence

We will define the target sequence length, the sequence we want to avoid interacting with, and the property targets for the optimizer.

In [ ]:
# Define parameters
sequence_length = 50  # Target length for the designed sequence

target_sequence = "MASNDYTQQATQSYGAYPTQPGQGYSQQSSQPYGQQSYSGYSQSTDTSGYGQSSYSSYGQSQNTGYGT"  # Example target sequence to avoid interacting with

# Set property targets
fraction_disorder_target = 1.0  # Fully disordered
self_epsilon_target = -10.0     # Strong self-attraction (negative value)
epsilon_by_value_target = 0.0   # No interaction with the target sequence

# You may adjust these values as needed for your use case.

## Initialize the SequenceOptimizer and Add Properties

We will create a `SequenceOptimizer` instance and add the three properties to optimize:

In [ ]:
# Initialize the optimizer
optimizer = SequenceOptimizer(target_length=sequence_length, max_iterations=2000)

# Add FractionDisorder property (maximize disorder)
optimizer.add_property(goose.FractionDisorder, target_value=fraction_disorder_target, weight=1.0)

# Add SelfEpsilon property (maximize self-attraction)
optimizer.add_property(goose.MeanSelfEpsilon, target_value=self_epsilon_target, weight=1.0, model='mpipi')

# Add EpsilonByValue property (minimize interaction with target_sequence)
optimizer.add_property(goose.MeanEpsilonWithTarget, target_value=epsilon_by_value_target, target_sequence=target_sequence, weight=1.0, model='mpipi')

## Run the Optimization

We will now run the optimizer to generate a sequence that meets the specified criteria.

In [ ]:
# Run the optimizer
designed_sequence = optimizer.run()

## Evaluate the Designed Sequence

Let's check the properties of the designed sequence to confirm it meets our goals:

In [ ]:
# Compute disorder fraction using metapredict
predicted_disorder = meta.predict_disorder(designed_sequence)
fraction_disorder = (predicted_disorder > 0.5).sum() / len(predicted_disorder)

# Compute self-epsilon and epsilon with target using finches (mpipi)
from finches.frontend.mpipi_frontend import Mpipi_frontend
model = Mpipi_frontend()

self_interaction = model.epsilon(designed_sequence, designed_sequence)
target_interaction = model.epsilon(designed_sequence, target_sequence)

print(f"Fraction Disorder: {fraction_disorder:.2f}")
print(f"Self Epsilon (should be strongly negative): {self_interaction:.2f}")
print(f"Epsilon with Target Sequence (should be near zero): {target_interaction:.2f}")